In [67]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/Users/addisonpelayo/Desktop/reviews_Patio_Lawn_and_Garden_5.json.gz')

In [68]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1JZFGZEZVWQPY,B00002N674,"Carter H ""1amazonreviewer@gmail . com""","[4, 4]",Good USA company that stands behind their prod...,4.0,Great Hoses,1308614400,"06 21, 2011"
1,A32JCI4AK2JTTG,B00002N674,"Darryl Bennett ""Fuzzy342""","[0, 0]",This is a high quality 8 ply hose. I have had ...,5.0,Gilmour 10-58050 8-ply Flexogen Hose 5/8-Inch ...,1402272000,"06 9, 2014"
2,A3N0P5AAMP6XD2,B00002N674,H B,"[2, 3]",It's probably one of the best hoses I've ever ...,4.0,Very satisfied!,1336176000,"05 5, 2012"
3,A2QK7UNJ857YG,B00002N674,Jason,"[0, 0]",I probably should have bought something a bit ...,5.0,Very high quality,1373846400,"07 15, 2013"
4,AS0CYBAN6EM06,B00002N674,jimmy,"[1, 1]",I bought three of these 5/8-inch Flexogen hose...,5.0,Good Hoses,1375660800,"08 5, 2013"
...,...,...,...,...,...,...,...,...,...
13267,AT53ZTTO707MB,B00KS0F4FI,I Do The Speed Limit,"[1, 2]",Simple. Perfect. Plenty big enough. Durable...,5.0,Great pair of claws,1403827200,"06 27, 2014"
13268,AYB4ELCS5AM8P,B00KS0F4FI,"John B. Goode ""JBG""","[0, 0]",These claws are fantastic. They are made of ha...,5.0,Sharp...,1405123200,"07 12, 2014"
13269,AZMY6E8B52L2T,B00KS0F4FI,"JP ""J.P.""","[1, 3]",I really like theseOuddy Heat Resistant Meat C...,5.0,Really Helpful...,1405123200,"07 12, 2014"
13270,AEC90GPFKLAAW,B00KS0F4FI,"Lisa Kearns ""Lisa Kearns""","[0, 0]",I make pulled pork in the crock pot pretty oft...,5.0,Shreds meat perfectly!,1405468800,"07 16, 2014"


In [69]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Count the number of times each rating appears in the dataset
data = df['overall'].value_counts().sort_index(ascending=False)

# Create the histogram
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} patio, lawn and garden ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [70]:
# Number of ratings per movie
data = df.groupby('asin')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per garden',
                   xaxis = dict(title = 'Number of Ratings Per Garden'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [71]:
# Number of ratings per user
data = df.groupby('reviewerID')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0, size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [72]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

In [73]:
import numpy as np
from sklearn.model_selection import train_test_split

# df2 = df.iloc[:30000, :]
# df2 = df2.groupby('reviewerID').filter(lambda x: len(x) >= 5)
data_train, data_test = train_test_split(df, test_size=0.20, random_state=42, stratify=df['reviewerID'])

In [74]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data_train_cf = Dataset.load_from_df(data_train[['reviewerID', 'asin', 'overall']], reader)
data_test_cf = Dataset.load_from_df(data_test[['reviewerID', 'asin', 'overall']], reader)

In [75]:
from surprise import KNNBaseline

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
knn = KNNBaseline(sim_options=sim_options)

In [76]:
from surprise.model_selection import cross_validate

results = cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0716  1.0832  1.0566  1.0705  0.0109  
Fit time          0.09    0.06    0.08    0.08    0.01    
Test time         0.07    0.07    0.14    0.09    0.03    


In [77]:
from surprise import NormalPredictor

cross_validate(NormalPredictor(), data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.4041  1.4171  1.3823  1.4012  0.0144  
Fit time          0.01    0.01    0.01    0.01    0.00    
Test time         0.03    0.02    0.02    0.03    0.00    


{'test_rmse': array([1.40413582, 1.41713734, 1.38229593]),
 'fit_time': (0.01346898078918457, 0.013476133346557617, 0.011932849884033203),
 'test_time': (0.03415513038635254,
  0.024823665618896484,
  0.022956132888793945)}

In [78]:
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [79]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible

class GroupAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # Return the baseline estimate
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return bsl


In [80]:
from surprise import SVDpp

# We'll use the famous SVD algorithm.
svd = SVDpp()

results = cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=False)

In [81]:
from surprise import SVD
from surprise import BaselineOnly
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), BaselineOnly(), CoClustering()]:
    
    print("Testing {}".format(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Testing <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fc7585ad650>
Testing <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7fc75bc4b990>
Testing <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7fc7439c9a10>
Testing <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7fc7439c97d0>
Testing <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7fc74161f850>
Testing <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7fc75cde9790>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Testing <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7fc74376b910>
Estimating biases using als...
Estimating biases using als

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.990275,1.414313,0.072275
BaselineOnly,0.993869,0.015190,0.016881
SVD,0.996130,0.466117,0.024606
KNNBaseline,1.073561,0.076690,0.162509
CoClustering,1.107380,0.348575,0.019335
SlopeOne,1.168458,0.036387,0.033404
NMF,1.198193,0.542307,0.019300
NormalPredictor,1.391469,0.010205,0.026634


In [82]:
# Execute KNN
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.fit(data.build_full_trainset())

# Target item to analyze its neighbourhood
garden_name = 'B00KS0F4FI'

# Get the closes neighbourds
neighbors = knn.get_neighbors(knn.trainset.to_inner_iid(garden_name), k=10)
# Translate the internal ids used in the algorithm to the movie names
neighbors = (knn.trainset.to_raw_iid(asin) for asin in neighbors)

print()
print('The 10 nearest neighbors of {} are:\n'.format(garden_name))
for asin in neighbors:
    print("\t",asin)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of B00KS0F4FI are:

	 B00J96ZBB4
	 B00JPWZY32
	 B00AA8WPGY
	 B000F0BVYE
	 B008RGCJ5I
	 B00IPL91M4
	 B000BWY8KK
	 B00C88DV6M
	 B000A0VOD2
	 B00008Z9ZG


In [83]:
garden_bias = [(b, data.build_full_trainset().to_raw_iid(i)) for i, b in enumerate(svd.bi)]
print("Top 15 gardens:")
sorted(garden_bias, key=lambda x: x[0])[:15]

Top 15 gardens:


[(-1.1472198346138023, 'B000WEOQV8'),
 (-1.1415475657706953, 'B000A1E690'),
 (-1.1100521984635014, 'B000GD3L0C'),
 (-1.100691810020144, 'B000H1W8FC'),
 (-1.0512478370333238, 'B00J96ZBB4'),
 (-1.0080871849879756, 'B00004SD72'),
 (-0.9887992327657904, 'B0007ZGUK2'),
 (-0.9558111783253785, 'B00002N6AN'),
 (-0.9416264895500175, 'B000EI8QUE'),
 (-0.9320074883285218, 'B003RDKKV6'),
 (-0.922876343871306, 'B002JUOWB2'),
 (-0.9134749831708324, 'B000EVE2ZE'),
 (-0.8912664182690753, 'B00004TBMV'),
 (-0.8910646109673245, 'B000NZZG3S'),
 (-0.8883462862191162, 'B000F97DYC')]

In [84]:
print("Top 15 gardens:")
sorted(garden_bias, key=lambda x: -x[0])[:15]

Top 15 gardens:


[(0.6288364725011645, 'B003959G9Y'),
 (0.615853379787031, 'B000BQPW6A'),
 (0.6058526263910817, 'B0007UQ2N2'),
 (0.5562974298719514, 'B000HHQAQY'),
 (0.5370288814636456, 'B000WEPFOA'),
 (0.5342957041995842, 'B000ZZU960'),
 (0.5213371738725929, 'B000W496PO'),
 (0.5199723948760979, 'B0012RS2I2'),
 (0.5157433023821215, 'B0006U2FJ8'),
 (0.5014155945985518, 'B000RYL1NE'),
 (0.5011868895576673, 'B00004RA0O'),
 (0.48736848700367696, 'B003TSE7A4'),
 (0.48461643297819545, 'B0031045XE'),
 (0.48369466519285403, 'B00008GS96'),
 (0.4825255488372925, 'B000EVE17S')]

In [85]:
user_bias = [(b, data.build_full_trainset().to_raw_uid(i)) for i, b in enumerate(svd.bu)]
sorted(user_bias, key=lambda x: x[0])[-1]

(0.722473917608561, 'A3919C6CEY5A3G')

In [86]:
df[df.reviewerID == 'A3919C6CEY5A3G']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
1231,A3919C6CEY5A3G,B0002IU8UW,NaN,"[0, 0]","as described, came quick, no issues",5.0,As described,1405382400,"07 15, 2014"
6776,A3919C6CEY5A3G,B002JUOWB2,NaN,"[0, 0]","Working Great. Wouldn't have minded handles, b...",5.0,Very happy with this,1405382400,"07 15, 2014"
7765,A3919C6CEY5A3G,B003MOD2HY,NaN,"[0, 0]",Very happy with this purchase. Expanded to be ...,5.0,Great purchase,1405382400,"07 15, 2014"
8267,A3919C6CEY5A3G,B004IAM29K,NaN,"[0, 0]",as described,5.0,As described,1405382400,"07 15, 2014"
13255,A3919C6CEY5A3G,B00JUQI4JO,andrew scott mallory,"[0, 0]",smaller then i thought but they are as describ...,5.0,As described,1405382400,"07 15, 2014"


In [87]:
sorted(user_bias, key=lambda x: x[0])[0]

(-1.3516171764204068, 'A2YB7DLC3FOR7W')

In [88]:
df[df.reviewerID == 'A2YB7DLC3FOR7W']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
822,A2YB7DLC3FOR7W,B00008GS96,"Pamela V ""&#34;MS V&#34;""","[3, 4]",I'm glad this gadget came with two double A ba...,4.0,Works fine - but fairly hard to handle!,1149811200,"06 9, 2006"
3969,A2YB7DLC3FOR7W,B000MV5V6A,"Pamela V ""&#34;MS V&#34;""","[1, 1]",Havahart Easy Set Live Animal Cage Trap is sim...,5.0,Humane Way to Relocate Destructive Wild Animals,1367280000,"04 30, 2013"
6180,A2YB7DLC3FOR7W,B001JJCW0M,"Pamela V ""&#34;MS V&#34;""","[2, 2]",The Victor M260 Multi-Kill electronic mouse tr...,2.0,Not working; the mice liveth!,1309737600,"07 4, 2011"
6440,A2YB7DLC3FOR7W,B001UTY5MY,"Pamela V ""&#34;MS V&#34;""","[0, 0]","I bought this as a gift for my mother, and she...",5.0,My Mom Loved It!,1387324800,"12 18, 2013"
9176,A2YB7DLC3FOR7W,B0060UO4SS,"Pamela V ""&#34;MS V&#34;""","[0, 1]",I use my Havahart 1064 Large 2-door easy set t...,4.0,"A bit fast on the trigger, but works as it should",1367798400,"05 6, 2013"
11283,A2YB7DLC3FOR7W,B00B7H7SRI,"Pamela V ""&#34;MS V&#34;""","[3, 4]","I live on 5 acres and have many bushes, plants...",5.0,Fantastic Set! More Power Than Expected!,1372636800,"07 1, 2013"
12842,A2YB7DLC3FOR7W,B00GSQ6HXO,"Pamela V ""&#34;MS V&#34;""","[1, 2]",Just this morning I sat on the steps of my fro...,5.0,Perfect Timing!,1398038400,"04 21, 2014"
12985,A2YB7DLC3FOR7W,B00GVL015Q,"Pamela V ""&#34;MS V&#34;""","[3, 3]",This is a lovely hummingbird feeder. It has a...,5.0,Cute Copper & Glass Hummingbird Feeder!,1398038400,"04 21, 2014"
